## Configuration
_Initial steps to get the notebook ready to play nice with our repository. Do not delete this section._

Code formatting with [black](https://pypi.org/project/nb-black/).

In [1]:
%load_ext lab_black

In [2]:
import os
import pytz
import glob
import pathlib

this_dir = pathlib.Path(os.path.abspath(""))
data_dir = this_dir / "data"

In [3]:
import requests
import pandas as pd
import numpy as np
from datetime import datetime

## Download

Retrieve the data for both Yuba and Sutter counties

In [4]:
url = "https://services5.arcgis.com/THtdW72WxYCCmIVL/ArcGIS/rest/services/CasesWeekly_Public/FeatureServer/0/query?where=1%3D1&objectIds=&time=&resultType=none&outFields=reportdt%2Cname%2CLiveOak%2CYubaCity%2CSutterOther%2CMarysville%2COlivehurstLinda%2CPlumasLake%2CWheatland%2CYubaOther&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnDistinctValues=false&cacheHint=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&sqlFormat=none&f=pjson&token="

In [5]:
r = requests.get(url)

In [6]:
data = r.json()

## Parse

In [7]:
dict_list = []

In [8]:
for item in data["features"]:
    d = dict(item["attributes"])
    dict_list.append(d)

In [9]:
df = pd.DataFrame(dict_list)

In [10]:
df = df[
    [
        "reportdt",
        "Name",
        "LiveOak",
        "YubaCity",
        "SutterOther",
        "Marysville",
        "OlivehurstLinda",
        "Wheatland",
        "YubaOther",
        "PlumasLake",
    ]
]

In [11]:
df = df.rename(columns={"Name": "name"})

Set the date

In [12]:
df["reportdt"] = df["reportdt"] / 1000

In [13]:
df["reportdt"] = pd.to_datetime(df["reportdt"], unit="s")

In [14]:
df["county_date"] = df["reportdt"].dt.date

Get the latest date in the timeseries

In [15]:
latest_df = df.loc[df.groupby(["name", "county_date"]).reportdt.idxmax()]

Sift out Sutter County's cities

In [16]:
sutter_df = latest_df[latest_df.name.eq("Sutter County")]

In [17]:
sutter_df = sutter_df[["LiveOak", "YubaCity", "SutterOther", "county_date", "reportdt"]]

In [18]:
sutter_latest = sutter_df[sutter_df.reportdt == sutter_df.reportdt.max()]

Melt down

In [19]:
sutter_latest.set_index("county_date", inplace=True)

In [20]:
sutter_melt = pd.melt(
    sutter_latest.reset_index(),
    id_vars="county_date",
    var_name="area",
    value_name="confirmed_cases",
)

Clean names

In [21]:
sutter_clean = sutter_melt[sutter_melt["area"] != "reportdt"]

In [22]:
clean_sutter_cities = {
    "YubaCity": "Yuba City",
    "YubaCIty": "Yuba City",
    "LiveOak": "Live Oak",
    "SutterOther": "Other",
}

In [23]:
sutter_clean = sutter_clean.replace({"area": clean_sutter_cities})

In [24]:
sutter_clean.insert(0, "county", "Sutter")
sutter_clean

,county,county_date,area,confirmed_cases


## Vet

In [25]:
try:
    assert not len(sutter_clean) > 3
except AssertionError:
    raise AssertionError("Sutter County's scraper has extra rows")

In [26]:
try:
    assert not len(sutter_clean) < 3
except AssertionError:
    raise AssertionError("Sutter County's scraper is missing rows")

AssertionError: Sutter County's scraper is missing rows

## Export

Set date

In [72]:
tz = pytz.timezone("America/Los_Angeles")

In [73]:
today = datetime.now(tz).date()

In [74]:
slug = "sutter"

In [75]:
sutter_clean.to_csv(data_dir / slug / f"{today}.csv", index=False)

## Combine

In [76]:
csv_list = [
    i
    for i in glob.glob(str(data_dir / slug / "*.csv"))
    if not str(i).endswith("timeseries.csv")
]

In [77]:
df_list = []
for csv in csv_list:
    if "manual" in csv:
        df = pd.read_csv(csv, parse_dates=["date"])
    else:
        file_date = csv.split("/")[-1].replace(".csv", "")
        df = pd.read_csv(csv, parse_dates=["county_date"])
        df["date"] = file_date
    df_list.append(df)

In [78]:
df = pd.concat(df_list).sort_values(["date", "area"])

In [79]:
df.to_csv(data_dir / slug / "timeseries.csv", index=False)